# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [97]:
# import libraries
import pyspark as spark
from pyspark.sql import SparkSession
import zipfile
import numpy as np
import plotly.graph_objects as go
from tqdm import tqdm

# unzip compressed data file
zip_ref = zipfile.ZipFile('mini_sparkify_event_data.json.zip', 'r')
zip_ref.extractall()
zip_ref.close()

In [100]:
# create a Spark session
spark = SparkSession.builder \
        .appName("Sparkify") \
        .getOrCreate()


# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [3]:
# load data set
data = spark.read.json("mini_sparkify_event_data.json")
data.show()
data.columns
data.count()

+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+---------------+-------------+---------+--------------------+------+-------------+--------------------+------+
|              artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|           page| registration|sessionId|                song|status|           ts|           userAgent|userId|
+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+---------------+-------------+---------+--------------------+------+-------------+--------------------+------+
|      Martha Tilston|Logged In|    Colin|     M|           50| Freeman|277.89016| paid|     Bakersfield, CA|   PUT|       NextSong|1538173362000|       29|           Rockpools|   200|1538352117000|Mozilla/5.0 (Wind...|    30|
|    Five Iron Frenzy|Logged In|    Micah|     M|           79|    Long|236.09424| free|Bost

286500

In [4]:
data.createOrReplaceTempView("data")

In [5]:
# clean data
# Try to figure out which columns have missing values, are NULL
#for column in in data.columns

print('Check if Null')
for column in data.columns:
    print(column,spark.sql(f"SELECT * FROM data WHERE {column} IS NULL").count())
print()
print('Check if empty string')
for column in data.columns:
    print(column,spark.sql(f"SELECT * FROM data WHERE {column} = ''").count())

Check if Null
artist 58392
auth 0
firstName 8346
gender 8346
itemInSession 0
lastName 8346
length 58392
level 0
location 8346
method 0
page 0
registration 8346
sessionId 0
song 58392
status 0
ts 0
userAgent 8346
userId 0

Check if empty string
artist 0
auth 0
firstName 0
gender 0
itemInSession 0
lastName 0
length 0
level 0
location 0
method 0
page 0
registration 0
sessionId 0
song 0
status 0
ts 0
userAgent 0
userId 8346


- No missing values for auth, itemInSession, level, method, page, sessionID, status, ts, userID
- but 8346 empty string userIds
We are not going to consider empty string userIds therefore we create a new data frame

In [101]:
# Create data frame without empty user ids 
data_cleaned = spark.sql(f"SELECT * FROM data WHERE userId != ''")
data_cleaned.createOrReplaceTempView("data_cleaned")

In [8]:
# Number of unique items in each row
for column in data_cleaned.columns:
    print(column,spark.sql(f"SELECT COUNT(DISTINCT({column})) FROM data_cleaned").collect()[0][0])

artist 17655
auth 2
firstName 189
gender 2
itemInSession 1311
lastName 173
length 14865
level 2
location 114
method 2
page 19
registration 225
sessionId 2312
song 58480
status 3
ts 269770
userAgent 56
userId 225


- So 225 unique users, that we have to make predictions
- Lets make a plot 

Let's dig in the data set a little bit more and investigate, how many paid users there are and if their interaction behavior differs

In [28]:
# Comparison of number of interactions
paid_user_interactions = np.array(data_cleaned[data_cleaned['level']=='paid'].groupBy('userId').count().orderBy('count', ascending=False).collect()).astype(int)
free_user_interactions = np.array(data_cleaned[data_cleaned['level']=='free'].groupBy('userId').count().orderBy('count', ascending=False).collect()).astype(int)

# Comparison number of paied and free users
fig = go.Figure()
fig.add_trace(go.Bar(y = [free_user_interactions.shape[0]], name = 'free'))
fig.add_trace(go.Bar(y = [paid_user_interactions.shape[0]], name = 'paid'))
fig.add_trace(go.Bar(y = [225], name = 'unique'))
fig.update_yaxes(title_text='Number of users')
fig.update_layout(title_text="Comparison of subscription levels",
                  template='plotly_white')
fig.show()
# Comparison of number of interactions
fig = go.Figure()
fig.add_trace(go.Histogram(x=free_user_interactions[:,1], name = 'free'))
fig.add_trace(go.Histogram(x=paid_user_interactions[:,1], name = 'paid'))
fig.update_xaxes(title_text='Number of interactions')
fig.update_yaxes(title_text='Number of users')
fig.update_layout(title_text="Comparison interactions free and paying users",
                  template='plotly_white')
fig.show()

- Of the 225 unique users, 165 at one time paid had a paid account and 195 a free account
- The interaction behavior differs strongly:
    - most of the free unsers have only up to a few hundrets (400) interactions
    - paying users have more than 1000

In [24]:
# Statistics about the number of interactions per session
free_interactions_per_session = np.array(data_cleaned[data_cleaned['level'] == 'free'].groupBy('sessionId').count().collect()).astype(int)
paid_interactions_per_session = np.array(data_cleaned[data_cleaned['level'] == 'paid'].groupBy('sessionId').count().collect()).astype(int)
fig = go.Figure()
fig.add_trace(go.Box(y=free_interactions_per_session[:,1], name = 'Free'))
fig.add_trace(go.Box(y=paid_interactions_per_session[:,1], name = 'Paid'))
fig.update_yaxes(title_text='Number of interactions per session')
fig.update_layout(title_text="Boxplot number of interactions per session",
                  template='plotly_white')
fig.show()

- Also the interaction behavior within a individual session differs:
    - Free users 99 % have between 1 and 148 inteactions with a median at 32
    - Paying users 99 % have between 1 and 413 interactions with a median at 89

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

We define the churn as interaction with the "Cancellation Confirmation" page.
To explore the data set for influencing factors, lets start by investigating the interactions with the different pages

In [26]:
# Count of the interactions with the different pages
free_count_of_pages = np.array(data_cleaned[data_cleaned['level'] == 'free'].groupBy('page').count().orderBy('count', ascending=False).collect())
paid_count_of_pages = np.array(data_cleaned[data_cleaned['level'] == 'paid'].groupBy('page').count().orderBy('count', ascending=False).collect())
fig = go.Figure()
fig.add_trace(go.Bar(x=free_count_of_pages[:,1].astype(float), y=free_count_of_pages[:,0], orientation='h', name='free'))
fig.add_trace(go.Bar(x=paid_count_of_pages[:,1].astype(float), y=paid_count_of_pages[:,0], orientation='h', name='paid'))
fig.update_xaxes(title_text='Number of interactions')
fig.update_layout(title_text="Number of page interactions",
                  height=600,
                  template='plotly_white')
fig.show()

So compared to the total interactions the interactions with the "Cancellation Confirmation" page is maginal. It is therefore more reasonable to investigate the interactions per unique user id

In [27]:
# Count of the interactions with the different pages for unique users per page
import pyspark.sql.functions as func
free_count_of_pages = np.array(data_cleaned[data_cleaned['level'] == 'free'].groupby('page').agg(func.countDistinct('userID')).collect())
paid_count_of_pages = np.array(data_cleaned[data_cleaned['level'] == 'paid'].groupby('page').agg(func.countDistinct('userID')).collect())


fig = go.Figure()
fig.add_trace(go.Bar(x=free_count_of_pages[:,1].astype(float), y=free_count_of_pages[:,0], orientation='h', name='unique free users'))
fig.add_trace(go.Bar(x=paid_count_of_pages[:,1].astype(float), y=paid_count_of_pages[:,0], orientation='h', name='unqiue paid users'))
fig.update_xaxes(title_text='Number of unique users')
fig.update_layout(title_text="Number of page interactions for unqiue users",
                  height=600,
                  template='plotly_white')
fig.show()


This representation is more meaningfull we see:
- Both free and paying users interacted withe the "Cancellation Confirmation" page
- Overall 52 users of the 225 unqiue users canceled the subscription

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

### Let's investigate some factors that might influence the cancellation. We are going to focus on:
- The attributes of the user e.g. gender
- The way the user interacts with the platform (which pages are visted, how much time is spend, time since registration...)
- If the interaction behavior changes
- If the user likes the content (Ratio of Thumbs Up to Thumps Down)
- If the user networks with the platform (adding friends)
- If errors of the plaform have an influence

In [33]:
# Comparison difference between last time stamp ts and time of registration for users how canceled users and non canceled users
canceled_users = np.array(spark.sql("SELECT userId, max(ts)-max(registration) FROM data_cleaned WHERE userId IN \
                        (SELECT userId FROM data_cleaned WHERE page = 'Cancellation Confirmation') \
                        GROUP BY userId").collect()).astype(int)
non_canceled_users = np.array(spark.sql("SELECT userId, max(ts)-max(registration) FROM data_cleaned WHERE userId NOT IN \
                        (SELECT userId FROM data_cleaned WHERE page = 'Cancellation Confirmation') \
                        GROUP BY userId").collect()).astype(int)

fig = go.Figure()
fig.add_trace(go.Box(y=canceled_users[:,1]/1000/3600/24, name = 'cancelling users'))  
fig.add_trace(go.Box(y=non_canceled_users[:,1]/1000/3600/24, name = 'non cancelling users')) 
fig.update_yaxes(title_text='Time difference [days]')
fig.update_layout(title_text="Difference between time of registration and last time stamp",
                  template='plotly_white')
fig.show()    
                  


- Non cancelling users are on average longer on the platform the platform with a median of 75.4 days since the registration
- Most of the users cancel after 51.3 days on the platform

In [34]:
# Distribution of gender of cancelling and non cancelling users
canceled_users = np.array(spark.sql("SELECT gender, COUNT(DISTINCT(userId)) FROM data_cleaned WHERE userId IN \
                        (SELECT userId FROM data_cleaned WHERE page = 'Cancellation Confirmation') \
                        GROUP BY gender").collect())
non_canceled_users = np.array(spark.sql("SELECT gender, COUNT(DISTINCT(userId)) FROM data_cleaned WHERE userId NOT IN \
                        (SELECT userId FROM data_cleaned WHERE page = 'Cancellation Confirmation') \
                        GROUP BY gender").collect())

from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2,
             subplot_titles=("Cancelling users", "Non cancelling users"),
             specs=[[{"type": "domain"}, {"type": "domain"}]])

fig.add_trace(
    go.Pie(values=canceled_users[:,1], labels=canceled_users[:,0], name='Cancelling users'),
    row=1, col=1
)

fig.add_trace(
    go.Pie(values=non_canceled_users[:,1], labels=non_canceled_users[:,0], name='Cancelling users'),
    row=1, col=2
)

fig.update_layout(title_text="Comparison of gender fraction",
                  template='plotly_white')
fig.show()

- The fraction of users with male gender is larger for cencelling users

In [40]:
# Interactions with pages
canceled_users = np.array(spark.sql("SELECT page, COUNT(DISTINCT(userId)) FROM data_cleaned WHERE userId IN \
                        (SELECT userId FROM data_cleaned WHERE page = 'Cancellation Confirmation') \
                        GROUP BY page").collect())
non_canceled_users = np.array(spark.sql("SELECT page, COUNT(DISTINCT(userId)) FROM data_cleaned WHERE userId NOT IN \
                        (SELECT userId FROM data_cleaned WHERE page = 'Cancellation Confirmation') \
                        GROUP BY page").collect())
num_cancels=52
num_non_cancels=173
fig = go.Figure()
fig.add_trace(go.Bar(x=canceled_users[:,1].astype(float)/num_cancels*100, y=canceled_users[:,0], orientation='h', name='cancelling users'))
fig.add_trace(go.Bar(x=non_canceled_users[:,1].astype(float)/num_non_cancels*100, y=non_canceled_users[:,0], orientation='h', name='non canceling users users'))
fig.update_xaxes(title_text='%')
fig.update_layout(title_text="Percentage of users visiting page to users",
                  height=600,
                  template='plotly_white')
fig.show()

- % of interactions with error page higher for non canceling users -> does not seem to influence cancellation
- Slightly more "Thumbs Down" and less "Thumbs Up" for cancelling users
- Non cancelling users add more friends


In [58]:
# Avg. length of interaction

canceled_users = np.array(spark.sql("SELECT userId, AVG(length) FROM data_cleaned WHERE userId IN \
                        (SELECT userId FROM data_cleaned WHERE page = 'Cancellation Confirmation') \
                        GROUP BY userId").collect())
non_canceled_users = np.array(spark.sql("SELECT userId, AVG(length) FROM data_cleaned WHERE userId NOT IN \
                        (SELECT userId FROM data_cleaned WHERE page = 'Cancellation Confirmation') \
                        GROUP BY userId").collect())

fig = go.Figure()
fig.add_trace(go.Box(y=canceled_users[:,1], name = 'cancelling users'))  
fig.add_trace(go.Box(y=non_canceled_users[:,1], name = 'non cancelling users')) 
fig.update_yaxes(title_text='Length of interaction [s]')
fig.update_layout(title_text="Comparison of avg. length of interaction",
                  template='plotly_white')
fig.show()

- Avg. length of interaction is very similar

In [113]:
# Change of time per session id
"""
We want to investigate the change of time spend per session, therefore we extract the sum of the lengths of all interactions or songs played for every session and take the last time stamp recorded for the id of the session. Then we calculate a simple linear fit for each user. If the gradient of the line is positive, the time spend per session is increasing and vice versa. The principle is shown for user 51 who canceled his subscription.
"""

canceled_user_ids =np.array(spark.sql("SELECT DISTINCT(userId) FROM data_cleaned WHERE userId IN \
                         (SELECT userId FROM data_cleaned WHERE page = 'Cancellation Confirmation')").collect())
non_canceled_user_ids =np.array(spark.sql("SELECT DISTINCT(userId) FROM data_cleaned WHERE userId NOT IN \
                         (SELECT userId FROM data_cleaned WHERE page = 'Cancellation Confirmation')").collect())

user = 51
arr = np.array(spark.sql(f"SELECT sessionId, SUM(length), MAX(ts)/1000/3600 FROM data_cleaned WHERE userId = {user} GROUP BY sessionId ORDER BY MAX(ts)").collect())

# Get time spend per individual session, calculate if positive or negative trend (linear regression)
model = np.polyfit(arr[:,2], arr[:,1], 1)
predict = np.poly1d(model)
x_pred = np.linspace(min(arr[:,2]), max(arr[:,2]),10)
y_pred = predict(x_pred)

fig = go.Figure()
fig.add_trace(go.Scatter(x=arr[:,2], y=arr[:,1], mode="lines+markers", name=f"Time spend per session user {user}"))
fig.add_trace(go.Scatter(x=x_pred, y=y_pred, mode="lines", name=f"Linear fit"))
fig.update_yaxes(title_text='Sum of lengths per interaction [s]')
fig.update_xaxes(title_text='Time stamp in [h]')
fig.update_layout(title_text=f"Change in time spend per session over time for user {user}",
                  template='plotly_white')
fig.show()


For this cancelling user the time spend per session is decreasing, the gradient of the line therefore negative. The gradients are now calculated for all individual users

In [193]:
# Calculate for all users
canceled_users_length_change = []
non_canceled_users_length_change = []

# Cancelling users

for user in tqdm(canceled_user_ids):
    user = int(user[0])
    arr = np.array(spark.sql(f"SELECT sessionId, SUM(length), MAX(ts)/1000/3600 FROM data_cleaned WHERE userId = {user} GROUP BY                                        sessionId ORDER BY MAX(ts)").collect())
    if len(arr)>1:
        model = np.polyfit(arr[:,2].astype(float), arr[:,1].astype(float), 1)
        canceled_users_length_change.append((user,model[0]))
    else:
        canceled_users_length_change.append((user,0))

# Non canceling users

for user in tqdm(non_canceled_user_ids):
    user = int(user[0])
    arr = np.array(spark.sql(f"SELECT sessionId, SUM(length), MAX(ts)/1000/3600 FROM data_cleaned WHERE userId = {user} GROUP BY                                        sessionId ORDER BY MAX(ts)").collect())
    if len(arr)>1:
        model = np.polyfit(arr[:,2].astype(float), arr[:,1].astype(float), 1)
        non_canceled_users_length_change.append((user,model[0]))
    else:
        non_canceled_users_length_change.append((user,0))

canceled_users_length_change = np.array(canceled_users_length_change)
non_canceled_users_length_change = np.array(non_canceled_users_length_change)
fig = go.Figure()
fig.add_trace(go.Box(y=canceled_users_length_change[:,1], name = 'cancelling users', ))  
fig.add_trace(go.Box(y=non_canceled_users_length_change[:,1], name = 'non cancelling users', ))
fig.update_yaxes(title_text='Trend of time spend per session')
fig.update_layout(title_text="Comparison of trend of time spend per session",
                  template='plotly_white')
fig.show()

100%|██████████| 173/173 [02:46<00:00,  1.04it/s]


In [166]:
# Change of length of interaction for last session
"""
We want to investigate the change of time spend per session item for the last session, therefore we extract lengths of all interactions or songs played for the session id with the highest time stamp. Then we calculate a simple linear fit for each user. If the gradient of the line is positive, the time spend per interaction is increasing and vice versa. The principle is shown for user 51 who canceled his subscription.
"""
user = 121
arr = np.array(spark.sql(f"SELECT sessionId, ts/1000/3600, length FROM data_cleaned WHERE userId = {user} AND \
            sessionId = (SELECT MAX(sessionId) FROM data_cleaned WHERE userId = {user}) AND length IS NOT NULL \
            ORDER BY ts DESC").collect())

# Get time spend per individual session, calculate if positive or negative trend (linear regression)
model = np.polyfit(arr[:,1], arr[:,2], 1)

predict = np.poly1d(model)
x_pred = np.linspace(min(arr[:,1]), max(arr[:,1]),10)
y_pred = predict(x_pred)

fig = go.Figure()
fig.add_trace(go.Scatter(x=arr[:,1], y=arr[:,2], mode="lines+markers", name=f"Time spend per session user {user}"))
fig.add_trace(go.Scatter(x=x_pred, y=y_pred, mode="lines", name=f"Linear fit"))
fig.update_yaxes(title_text='Sum of lengths per interaction [s]')
fig.update_xaxes(title_text='Time stamp in [h]')
fig.update_layout(title_text=f"Change in time spend per session over time for user {user}",
                  template='plotly_white')
fig.show()

[ 7.68211971e-01 -3.28324505e+05]


In [194]:
# Calculate for all users
canceled_users_length_change_last_sess = []
non_canceled_users_length_change_last_sess = []

# Cancelling users

for user in tqdm(canceled_user_ids):
    user = int(user[0])
    arr = np.array(spark.sql(f"SELECT sessionId, ts/1000/3600, length FROM data_cleaned WHERE userId = {user} AND \
            sessionId = (SELECT MAX(sessionId) FROM data_cleaned WHERE userId = {user}) AND length IS NOT NULL \
            ORDER BY ts DESC").collect())
    if len(arr)>1:
        model = np.polyfit(arr[:,1].astype(float), arr[:,2].astype(float), 1)
        canceled_users_length_change_last_sess.append((user,model[0]))
    else:
        canceled_users_length_change_last_sess.append((user,0))

# Non canceling users

for user in tqdm(non_canceled_user_ids):
    user = int(user[0])
    arr = np.array(spark.sql(f"SELECT sessionId, ts/1000/3600, length FROM data_cleaned WHERE userId = {user} AND \
            sessionId = (SELECT MAX(sessionId) FROM data_cleaned WHERE userId = {user}) AND length IS NOT NULL \
            ORDER BY ts DESC").collect())
    if len(arr)>1:
        model = np.polyfit(arr[:,1].astype(float), arr[:,2].astype(float), 1)
        non_canceled_users_length_change_last_sess.append((user,model[0]))
    else:
        non_canceled_users_length_change_last_sess.append((user,0))

canceled_users_length_change_last_sess = np.array(canceled_users_length_change_last_sess)
non_canceled_users_length_change_last_sess = np.array(non_canceled_users_length_change_last_sess)
fig = go.Figure()
fig.add_trace(go.Box(y=canceled_users_length_change_last_sess[:,1], name = 'cancelling users', ))  
fig.add_trace(go.Box(y=non_canceled_users_length_change_last_sess[:,1], name = 'non cancelling users', ))
fig.update_yaxes(title_text='Trend of time spend per interation')
fig.update_layout(title_text="Comparison of trend of time spend per interaction in last session",
                  template='plotly_white')
fig.show()

100%|██████████| 173/173 [06:37<00:00,  2.30s/it]


## Chosen features:
- ### Time difference between registration and last time stamp:
    To consider influence of time since registration
- ### Gender
    To consider influence of gender
- ### Radio of Thumps up to thumps down
    To consider overall liking of the offered music and the mood
- ### Number of Friends
    To consider the network aspect of the platform
- ### Number of interactions with error page
    To consider effect of malfunctions of the plaform
- ### Trend of time spend per session
    To consider decrease of long-term intrest in the platform for users with a lot of interactions
- ### Trend time spend per interaction
    To consider decrease of short-term intrest in the platform e.g. users that only use the platform for 1 or two session

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

In [35]:
# Add Churn column
def valueToCategory(value):
   if   value == 'Cancellation Confirmation': return 1
   else : return 0
# Calculate time between registration and timestamp
def timeDifference(registration,ts):
    return float(ts)-float(registration)
# Calculate number of interactions with error page for users
def errorPageInteractions(userId):
    return data_feat_eng[(data_feat_eng['page']=='Error') & (data_feat_eng['userId']==userId)].count()
# Calculate avg. number of Session items
# Calculate avg. session length

from pyspark.sql.functions import udf
from pyspark.sql.types import *

timeDifference = udf(timeDifference, FloatType())
data_feat_eng = data_cleaned.withColumn("Churn", udfValueToCategory('page'))
data_feat_eng = data_cleaned.withColumn("TimeDifference", timeDifference('registration','ts'))
data_feat_eng.show()

+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+---------------+-------------+---------+--------------------+------+-------------+--------------------+------+--------------+
|              artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|           page| registration|sessionId|                song|status|           ts|           userAgent|userId|TimeDifference|
+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+---------------+-------------+---------+--------------------+------+-------------+--------------------+------+--------------+
|      Martha Tilston|Logged In|    Colin|     M|           50| Freeman|277.89016| paid|     Bakersfield, CA|   PUT|       NextSong|1538173362000|       29|           Rockpools|   200|1538352117000|Mozilla/5.0 (Wind...|    30|  1.78755008E8|
|    Five Iron Frenzy|Logged In|

In [313]:
# Create data frame with chose features
canceled_users_length_change = np.nan_to_num(canceled_users_length_change)
non_canceled_users_length_change = np.nan_to_num(non_canceled_users_length_change)
canceled_users_length_change_last_sess = np.nan_to_num(canceled_users_length_change_last_sess)
non_canceled_users_length_change_last_sess = np.nan_to_num(non_canceled_users_length_change_last_sess)

data_feat_eng = spark.sql(""" SELECT
                 userId, 
                 (MAX(ts)-MIN(registration))/1000/3600 AS time_since_reg,
                 MAX(CASE WHEN gender = 'M' Then 1 ELSE 0 END) AS gender,
                 SUM(CASE WHEN page = 'Add Friend' THEN 1 ELSE 0 END) AS num_friends,
                 NVL(SUM(CASE WHEN page = 'Thumbs Up' THEN 1 ELSE 0 END)/SUM(CASE WHEN page = 'Thumbs Down' THEN 1 ELSE 0 END),0) AS TU_TD_ratio,
                 SUM(CASE WHEN page = 'Error' THEN 1 ELSE 0 END) AS error_page_inter,
                 MAX(CASE WHEN page = 'Cancellation Confirmation' THEN 1 ELSE 0 END) AS churn
                 FROM data_cleaned GROUP BY userId
                """)

def length_trend(value):
    if float(value) in canceled_users_length_change[:, 0]: return float(round(canceled_users_length_change[np.where(canceled_users_length_change[:, 0]==float(value))[0][0],1],2))
    elif float(value) in non_canceled_users_length_change[:, 0]: return float(round(non_canceled_users_length_change[np.where(non_canceled_users_length_change[:, 0]==float(value))[0][0],1],2))
    else: None
def length_trend_last_session(value):
    if float(value) in canceled_users_length_change_last_sess[:, 0]: return float(round(canceled_users_length_change_last_sess[np.where(canceled_users_length_change_last_sess[:, 0]==float(value))[0][0],1],2))
    elif float(value) in non_canceled_users_length_change_last_sess[:, 0]: return float(round(non_canceled_users_length_change_last_sess[np.where(non_canceled_users_length_change_last_sess[:, 0]==float(value))[0][0],1],2))
    else: None


from pyspark.sql.functions import udf
from pyspark.sql.types import *
length_trend = udf(length_trend, StringType())
length_trend_last_session = udf(length_trend_last_session, StringType())
data_feat_eng = data_feat_eng.withColumn("length_change", length_trend('userId'))
data_feat_eng = data_feat_eng.withColumn("length_change_last_sess", length_trend_last_session('userId'))
# reorder columns
data_feat_eng = data_feat_eng.select("userId", "time_since_reg", "gender", "num_friends", "TU_TD_ratio" , "error_page_inter", "length_change", "length_change_last_sess", "churn")

In [314]:
data_feat_eng.show()

+------+------------------+------+-----------+------------------+----------------+-------------+-----------------------+-----+
|userId|    time_since_reg|gender|num_friends|       TU_TD_ratio|error_page_inter|length_change|length_change_last_sess|churn|
+------+------------------+------+-----------+------------------+----------------+-------------+-----------------------+-----+
|100010|1335.4477777777777|     0|          4|               3.4|               0|         5.74|                   8.29|    0|
|200002|1681.7911111111111|     1|          4|               3.5|               0|        26.01|                  -4.63|    0|
|   125|1711.6052777777777|     1|          0|               0.0|               0|          0.0|                 -53.93|    1|
|    51| 466.9402777777778|     1|         28| 4.761904761904762|               1|      -165.54|                  -1.16|    1|
|   124|3157.3419444444444|     0|         74| 4.170731707317073|               6|          0.0|               

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

In [322]:
# Import mllip packages
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, CrossValidatorModel
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier
from pyspark.ml.feature import VectorAssembler, Normalizer
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [317]:
# Shuffle data
cv_data, hold_off_data= data_feat_eng.randomSplit([0.75, 0.25], seed=42)

# Split data in data for cross validation and hold-off data
cv_data.show()

+------+------------------+------+-----------+------------------+----------------+-------------+-----------------------+-----+
|userId|    time_since_reg|gender|num_friends|       TU_TD_ratio|error_page_inter|length_change|length_change_last_sess|churn|
+------+------------------+------+-----------+------------------+----------------+-------------+-----------------------+-----+
|100010|1335.4477777777777|     0|          4|               3.4|               0|         5.74|                   8.29|    0|
|   125|1711.6052777777777|     1|          0|               0.0|               0|          0.0|                 -53.93|    1|
|   124|3157.3419444444444|     0|         74| 4.170731707317073|               6|          0.0|                    0.0|    0|
|     7|1746.6763888888888|     1|          1|               7.0|               1|        -0.79|                 350.56|    0|
|    54|2658.0405555555553|     0|         33| 5.620689655172414|               1|          0.0|               

In [ ]:
clf = LogisticRegression()

cv_grid = ParamGridBuilder() \
    .addGrid(lr.elasticNetParam,[0.0, 0.1]) \
    .addGrid(lr.regParam,[0.0, 0.1]) \
    .build()

cross_val =CrossValidator(esimator = clf,
                          estimatorParamMaps = cv_grid,
                          evaluator = ,
                          numFolds = 3)

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.